In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import dggs
from dggs import DGGS, io as dggs_io, uitools
import dggs.datasets as data
import dggs.applib

### Load canberra map for data vis

This is a raster of google map, it was converted to DGGS raster and stored in HDF5 file.

In [ ]:
cbr_map = dggs_io.h5_load('cbr-google-map.h5')

### Get some spatial queries

1. Load named polygon from shape file
2. Convert polygon to rHealPix
3. Rasterise at the specified DGGS scale
4. Convert raster to minimal textual representation

One can combine two or more queries by just concatenating them, this will produce correct but not optimal query. One can however simplify the combined query (currently only naive version of simplification is implemented).

In [ ]:
mm = data.act_suburb_mask(12, mode='text')

q1 = mm('BRADDON')
q2 = mm('TURNER')

print('Braddon')
display(uitools.addr_mask_repr(q1))
print('Turner')
display(uitools.addr_mask_repr(q2))

query = q1 + q2
print('Turner + Braddon')
display(uitools.addr_mask_repr(query))

query = dggs.simplify_mask_naive(query)

print('Turner + Braddon (simplified)')
display(uitools.addr_mask_repr(query))

In [ ]:
fig = plt.figure(figsize=(14,7))
roi = DGGS.roi_from_points(query)

ax1 = uitools.DgDraw(fig.add_axes([0,0,0.49,1]))
ax2 = uitools.DgDraw(fig.add_axes([0.5,0,0.49,1]))
ax1.raw.set_title('Raw')
ax1.imshow(cbr_map)
ax1.hide_axis()
ax1.qshow(q1,max_level=11)
ax1.qshow(q2,max_level=11)
ax1.zoom(roi);

ax2.raw.set_title('Simplified')
ax2.imshow(cbr_map)
ax2.qshow(query, max_level=11)
ax2.hide_axis()
ax2.zoom(roi);

### Load NEXIS Point Data

In [ ]:
df, crs = data.act_nexis()
df.head(3)

### Assign DGGS addresses to points

In [ ]:
df = dggs.pd_compute_address(df, 15, crs=crs)
df.head(3)

### Apply spatial query

Find points that fall within the query region. Currently only a naive version of the query is implemented. Computational complexity for naive implementation is $\Theta(NM)$, where $N$ is number of data points and $M$ is number of query cells. Proper implementation should probably be $\Theta(M \log N)$, or maybe even just $\Theta(M)$, assuming $\Theta(N \log N)$ preprocessing step is applied to the source data first. 

In [ ]:
mm = dggs.applib.pd_naive_overlap(query, df)
df_ = df[mm]

In [ ]:
ax = uitools.DgDraw(plt.figure(figsize=(12,8)))

ax.imshow(cbr_map)
ax.qshow(query, max_level=11)
ax.scatter(df.addr, color='b', marker='s', s=20, alpha=0.1)
ax.scatter(df_.addr, color='c', marker='.', s=20)
ax.roi(roi, 'y-', linewidth=2)
ax.hide_axis();

In [ ]:
ax.zoom(roi)
ax.figure

### Do some stats on a subset

In [ ]:
totals = df_['value people footprint addresses'.split()].sum()

print(f'''
People     {totals.people:,.0f}
Dwellings  {totals.addresses:,.0f}
Floor      {totals.footprint:,.0f} m2
Value      ${totals.value:,.0f}
''')